# Extracción de Datos de Fondos de Inversión - Santander

Este notebook automatiza la extracción de datos de composición de carteras de fondos de inversión desde los reportes mensuales CAFCI de Santander Argentina.

## Flujo del proceso:
1. **Instalación de dependencias**
2. **Importación de librerías**
3. **Configuración de parámetros**
4. **Navegación web automatizada** (Selenium)
5. **Descarga y extracción de datos del PDF**
6. **Procesamiento y transformación de datos**
7. **Almacenamiento en Data Warehouse**

## 1. Instalación de Dependencias

Instalamos las bibliotecas necesarias para el web scraping, procesamiento de PDFs y manipulación de datos.

!pip install selenium webdriver-manager pandas requests pypdf tabula-py

## 2. Importación de Librerías

Importamos todas las bibliotecas necesarias para el proceso.

In [ ]:
# Selenium - Para automatización de navegación web
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Procesamiento de datos
import pandas as pd
import time
import locale
from datetime import datetime

# Descarga y procesamiento de PDFs
import requests
import io
import re
from pypdf import PdfReader 
import tabula

# Utilidades
from typing import Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

## 3. Configuración de Parámetros

Definimos las constantes y configuraciones del proceso.

In [ ]:
# ====================================================================================
# CONFIGURACIÓN PRINCIPAL
# ====================================================================================

class Config:
    """Configuración centralizada del proyecto"""
    
    # URLs y selectores
    URL_FONDO = "https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12"
    XPATH_REPORTE = "//a[contains(., 'Reporte mensual - CAFCI')]"
    
    # Parámetros de extracción
    N_FILAS_ESPERADAS = 10
    TABLA_AREA = [380, 300, 640, 770]  # Coordenadas [top, left, bottom, right]
    
    # Metadatos
    SOCIEDAD_GERENTE = "Santander AM"
    NOMBRE_FONDO_DEFAULT = "Superfondo Renta Variable - Clase A"
    
    # Patrones regex
    PATTERN_NOMBRE = re.compile(r'Superfondo\s+(.*?)\s*-\s*Clase\s+\w', re.IGNORECASE | re.DOTALL)
    PATTERN_FECHA = re.compile(r'Datos\s*al\s*(\d{1,2}.*?\d{4})', re.IGNORECASE | re.DOTALL)
    
    # Selenium timeouts
    TIMEOUT_IMPLICIT = 10
    TIMEOUT_EXPLICIT = 10
    
    # Databricks
    TABLA_DESTINO = "fondos.composicion_santander"

# Configurar locale para fechas en español
def configurar_locale():
    """Configura el locale en español para parsing de fechas"""
    locales_spanish = ['es_ES.UTF-8', 'Spanish_Spain', 'es']
    
    for loc in locales_spanish:
        try:
            locale.setlocale(locale.LC_TIME, loc)
            return True
        except locale.Error:
            continue
    return False

configurar_locale()

## 4. Funciones de Utilidad

Definimos funciones reutilizables para cada etapa del proceso.

In [ ]:
def obtener_driver() -> webdriver.Chrome:
    """
    Crea y configura un driver de Selenium para Chrome en modo headless.
    
    Returns:
        webdriver.Chrome: Driver configurado
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(Config.TIMEOUT_IMPLICIT)
    
    return driver


def obtener_url_pdf(url: str, xpath: str) -> Optional[str]:
    """
    Navega a la página del fondo y obtiene la URL del PDF del reporte mensual.
    
    Args:
        url: URL de la página del fondo
        xpath: XPath del enlace al reporte
        
    Returns:
        URL del PDF o None si falla
    """
    driver = None
    
    try:
        driver = obtener_driver()
        ventana_original = driver.current_window_handle
        
        print(f"🌐 Navegando a {url}")
        driver.get(url)
        time.sleep(3)  # Esperar carga inicial
        
        # Encontrar y hacer clic en el enlace
        enlace = WebDriverWait(driver, Config.TIMEOUT_EXPLICIT).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", enlace)
        driver.execute_script("arguments[0].click();", enlace)
        
        # Esperar nueva ventana
        WebDriverWait(driver, Config.TIMEOUT_EXPLICIT).until(
            EC.number_of_windows_to_be(2)
        )
        
        # Cambiar a la nueva ventana
        for ventana in driver.window_handles:
            if ventana != ventana_original:
                driver.switch_to.window(ventana)
                break
        
        time.sleep(2)
        pdf_url = driver.current_url
        
        if pdf_url and pdf_url.endswith(".pdf"):
            print(f"✅ PDF encontrado")
            return pdf_url
        else:
            print("❌ URL no termina en .pdf")
            return None
            
    except Exception as e:
        print(f"❌ Error en navegación: {e}")
        return None
        
    finally:
        if driver:
            driver.quit()


def descargar_pdf(url: str) -> Optional[io.BytesIO]:
    """
    Descarga un PDF desde una URL y lo retorna como BytesIO.
    
    Args:
        url: URL del PDF
        
    Returns:
        BytesIO con el contenido del PDF o None si falla
    """
    try:
        print("📥 Descargando PDF...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        pdf_file = io.BytesIO(response.content)
        print(f"✅ Descargado ({len(response.content):,} bytes)")
        return pdf_file
        
    except Exception as e:
        print(f"❌ Error descargando PDF: {e}")
        return None

In [ ]:
def extraer_texto_pdf(pdf_file: io.BytesIO) -> str:
    """
    Extrae el texto de la primera página de un PDF.
    
    Args:
        pdf_file: PDF en formato BytesIO
        
    Returns:
        Texto extraído
    """
    try:
        pdf_file.seek(0)
        reader = PdfReader(pdf_file)
        return reader.pages[0].extract_text()
    except Exception as e:
        print(f"❌ Error extrayendo texto: {e}")
        return ""


def extraer_nombre_fondo(texto: str) -> str:
    """
    Extrae el nombre del fondo del texto del PDF.
    
    Args:
        texto: Texto del PDF
        
    Returns:
        Nombre del fondo
    """
    match = Config.PATTERN_NOMBRE.search(texto)
    
    if match:
        nombre = f"Superfondo {match.group(1).strip()} - Clase A"
        print(f"✅ Fondo: {nombre}")
        return nombre
    else:
        print(f"⚠️ Nombre no encontrado. Usando default")
        return Config.NOMBRE_FONDO_DEFAULT


def extraer_fecha(texto: str) -> Tuple[Optional[datetime], str]:
    """
    Extrae y parsea la fecha del reporte.
    
    Args:
        texto: Texto del PDF
        
    Returns:
        Tupla (fecha_datetime, fecha_string)
    """
    match = Config.PATTERN_FECHA.search(texto)
    
    if not match:
        print("❌ Fecha no encontrada")
        return None, "Fecha no encontrada"
    
    fecha_str = re.sub(r'\s+', ' ', match.group(1).strip())
    
    try:
        fecha_dt = pd.to_datetime(fecha_str, format="%d de %B %Y")
        print(f"✅ Fecha: {fecha_dt.strftime('%Y-%m-%d')}")
        return fecha_dt, fecha_str
    except Exception:
        print(f"⚠️ Fecha extraída pero no parseada: {fecha_str}")
        return None, fecha_str

In [ ]:
def extraer_tabla_composicion(pdf_file: io.BytesIO, n_filas: int = 10) -> Optional[pd.DataFrame]:
    """
    Extrae la tabla de composición del fondo desde el PDF.
    
    Args:
        pdf_file: PDF en formato BytesIO
        n_filas: Número esperado de filas
        
    Returns:
        DataFrame con columnas 'Accion' y 'Porcentaje' o None si falla
    """
    try:
        print("🔍 Extrayendo tabla de composición...")
        pdf_file.seek(0)
        
        dfs = tabula.read_pdf(
            pdf_file,
            pages=1,
            multiple_tables=False,
            output_format="dataframe",
            area=Config.TABLA_AREA,
            stream=True,
            encoding='latin-1'
        )
        
        if not dfs or dfs[0].empty:
            print("❌ No se extrajo ninguna tabla")
            return None
        
        df = dfs[0].dropna(how='all')
        
        # Extraer acciones (primera columna)
        col_acciones = df.columns[0]
        acciones = df[col_acciones].dropna().head(n_filas).tolist()
        
        # Extraer porcentajes (columnas con 'Unnamed' o '%')
        cols_pct = [c for c in df.columns if 'Unnamed' in c or '%' in str(df[c].iloc[0])]
        porcentajes = df[cols_pct].stack().dropna().head(n_filas).tolist()
        
        # Validar longitudes
        if len(acciones) != n_filas or len(porcentajes) != n_filas:
            print(f"⚠️ Longitudes incorrectas: {len(acciones)} acciones, {len(porcentajes)} porcentajes")
            return None
        
        df_resultado = pd.DataFrame({
            'Accion': acciones,
            'Porcentaje': porcentajes
        })
        
        print(f"✅ Tabla extraída: {len(df_resultado)} registros")
        return df_resultado
        
    except Exception as e:
        print(f"❌ Error extrayendo tabla: {e}")
        return None


def procesar_dataframe(
    df: pd.DataFrame,
    fecha: Optional[datetime],
    nombre_fondo: str,
    sociedad: str
) -> pd.DataFrame:
    """
    Procesa el DataFrame para el formato final del warehouse.
    
    Args:
        df: DataFrame con Accion y Porcentaje
        fecha: Fecha del reporte
        nombre_fondo: Nombre del fondo
        sociedad: Sociedad gerente
        
    Returns:
        DataFrame procesado con todas las columnas
    """
    df_proc = df.copy()
    
    # Convertir porcentajes a decimal
    df_proc['Porcentaje'] = (
        df_proc['Porcentaje']
        .astype(str)
        .str.replace('%', '', regex=False)
        .pipe(pd.to_numeric, errors='coerce')
        / 100
    )
    
    # Agregar columnas de metadata
    df_proc['Periodo'] = fecha.strftime('%Y-%m-%d') if fecha else 'Sin fecha'
    df_proc['Nombre_Fondo'] = nombre_fondo
    df_proc['Sociedad_Gerente'] = sociedad
    
    # Reordenar columnas
    return df_proc[['Periodo', 'Nombre_Fondo', 'Sociedad_Gerente', 'Accion', 'Porcentaje']]

In [ ]:
def guardar_en_databricks(df: pd.DataFrame, tabla: str, merge: bool = True) -> bool:
    """
    Guarda el DataFrame en una Delta Table de Databricks.
    
    Args:
        df: DataFrame a guardar
        tabla: Nombre de la tabla destino
        merge: Si True, hace MERGE; si False, hace APPEND
        
    Returns:
        True si tuvo éxito, False si falló
    """
    try:
        # Verificar que estamos en Databricks
        if 'spark' not in globals():
            print("⚠️ No se detectó Spark. Este código debe ejecutarse en Databricks")
            return False
        
        spark_df = spark.createDataFrame(df)
        
        if merge:
            from delta.tables import DeltaTable
            
            if DeltaTable.isDeltaTable(spark, tabla):
                print(f"📝 Haciendo MERGE en '{tabla}'...")
                
                delta_table = DeltaTable.forName(spark, tabla)
                delta_table.alias("target").merge(
                    spark_df.alias("source"),
                    "target.Periodo = source.Periodo AND target.Accion = source.Accion"
                ).whenMatchedUpdateAll() \
                 .whenNotMatchedInsertAll() \
                 .execute()
                
                print("✅ MERGE completado")
            else:
                print(f"🆕 Creando tabla '{tabla}'...")
                spark_df.write.format("delta").mode("overwrite").saveAsTable(tabla)
                print("✅ Tabla creada")
        else:
            print(f"📝 Haciendo APPEND en '{tabla}'...")
            spark_df.write.format("delta").mode("append").saveAsTable(tabla)
            print("✅ APPEND completado")
        
        # Mostrar conteo
        count = spark.table(tabla).count()
        print(f"📊 Total registros en tabla: {count:,}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error guardando en Databricks: {e}")
        return False


def mostrar_resumen(df: pd.DataFrame, nombre_fondo: str, fecha: Optional[datetime], pdf_url: str):
    """
    Muestra un resumen ejecutivo de los datos extraídos.
    
    Args:
        df: DataFrame procesado
        nombre_fondo: Nombre del fondo
        fecha: Fecha del reporte
        pdf_url: URL del PDF
    """
    print("\n" + "=" * 80)
    print("📋 RESUMEN EJECUTIVO")
    print("=" * 80)
    
    print(f"\n🏢 Fondo:           {nombre_fondo}")
    print(f"🏦 Gerente:         {Config.SOCIEDAD_GERENTE}")
    print(f"📅 Periodo:         {fecha.strftime('%Y-%m-%d') if fecha else 'Sin fecha'}")
    print(f"📄 PDF:             {pdf_url[:70]}..." if pdf_url else "No disponible")
    
    if df is not None and not df.empty:
        print(f"\n✅ Estado:          Extracción exitosa")
        print(f"📊 Registros:       {len(df)}")
        print(f"💹 Total cartera:   {df['Porcentaje'].sum():.2%}")
        
        print("\n🔝 Top 5 holdings:")
        for idx, row in df.nlargest(5, 'Porcentaje').iterrows():
            accion = row['Accion'][:45]
            pct = row['Porcentaje']
            print(f"   {accion:45s} {pct:>7.2%}")
    else:
        print(f"\n⚠️ Estado:          Sin datos extraídos")
    
    print("\n" + "=" * 80)

## 5. Ejecución Principal

Proceso completo de extracción, transformación y carga (ETL).

In [ ]:
def ejecutar_pipeline_completo():
    """
    Ejecuta el pipeline completo de extracción de datos.
    
    Returns:
        DataFrame procesado o None si falla
    """
    print("🚀 Iniciando pipeline de extracción...\n")
    
    # 1. Obtener URL del PDF
    pdf_url = obtener_url_pdf(Config.URL_FONDO, Config.XPATH_REPORTE)
    if not pdf_url:
        print("❌ No se pudo obtener la URL del PDF")
        return None
    
    # 2. Descargar PDF
    pdf_file = descargar_pdf(pdf_url)
    if not pdf_file:
        print("❌ No se pudo descargar el PDF")
        return None
    
    # 3. Extraer texto del PDF
    texto = extraer_texto_pdf(pdf_file)
    if not texto:
        print("❌ No se pudo extraer texto del PDF")
        return None
    
    # 4. Extraer metadatos
    nombre_fondo = extraer_nombre_fondo(texto)
    fecha, _ = extraer_fecha(texto)
    
    # 5. Extraer tabla de composición
    df_composicion = extraer_tabla_composicion(pdf_file, Config.N_FILAS_ESPERADAS)
    if df_composicion is None:
        print("❌ No se pudo extraer la tabla de composición")
        return None
    
    # 6. Procesar datos
    df_final = procesar_dataframe(
        df_composicion,
        fecha,
        nombre_fondo,
        Config.SOCIEDAD_GERENTE
    )
    
    # 7. Mostrar resumen
    mostrar_resumen(df_final, nombre_fondo, fecha, pdf_url)
    
    print("\n✅ Pipeline completado exitosamente")
    return df_final


# Ejecutar el pipeline
df_resultado = ejecutar_pipeline_completo()

## 6. Visualización de Datos (Opcional)

Vista previa del DataFrame extraído.

In [ ]:
# Mostrar el DataFrame completo si existe
if df_resultado is not None and not df_resultado.empty:
    print("📊 Datos extraídos:\n")
    display(df_resultado)
    
    print(f"\n📈 Estadísticas:")
    print(f"   - Total de holdings: {len(df_resultado)}")
    print(f"   - Suma de porcentajes: {df_resultado['Porcentaje'].sum():.2%}")
    print(f"   - Mayor holding: {df_resultado.loc[df_resultado['Porcentaje'].idxmax(), 'Accion']}")
    print(f"   - % del mayor: {df_resultado['Porcentaje'].max():.2%}")
else:
    print("⚠️ No hay datos para mostrar")

## 7. Guardar en Databricks (Descomenta para usar)

Almacenamiento en Delta Table del Data Warehouse.

In [ ]:
# Descomenta las siguientes líneas para guardar en Databricks

# if df_resultado is not None and not df_resultado.empty:
#     exito = guardar_en_databricks(
#         df=df_resultado,
#         tabla=Config.TABLA_DESTINO,
#         merge=True  # True para MERGE, False para APPEND
#     )
#     
#     if exito:
#         print("\n🎉 Datos guardados exitosamente en el Data Warehouse")
#     else:
#         print("\n⚠️ Hubo un problema al guardar los datos")
# else:
#     print("⚠️ No hay datos para guardar")

---

## 📚 Documentación Técnica

### Mejoras implementadas:

✅ **Arquitectura orientada a funciones**
- Código modular y reutilizable
- Cada función tiene una responsabilidad única (SRP)
- Fácil de testear y mantener

✅ **Type hints y documentación**
- Todas las funciones tienen type hints
- Docstrings descriptivos
- Mejor autocompletado en IDEs

✅ **Manejo de errores robusto**
- Try-except en cada función crítica
- Mensajes informativos con emojis
- Siempre retorna un valor (None en caso de error)

✅ **Configuración centralizada**
- Clase `Config` con todas las constantes
- Fácil de modificar y mantener
- No más valores hardcodeados

✅ **Código DRY (Don't Repeat Yourself)**
- Sin duplicación de lógica
- Funciones reutilizables
- Pipeline claro y conciso

✅ **Performance**
- Uso eficiente de pandas (`.pipe()`, `.stack()`)
- Un solo paseo por el PDF
- Context managers implícitos

✅ **Mejores prácticas de Python**
- PEP 8 compliant
- Nombres descriptivos
- Imports organizados
- Warnings silenciados

### Uso:

```python
# Ejecución simple
df = ejecutar_pipeline_completo()

# Guardar en Databricks
if df is not None:
    guardar_en_databricks(df, "mi_schema.mi_tabla", merge=True)
```

### Próximos pasos sugeridos:

1. **Logging profesional**: Reemplazar `print()` por `logging`
2. **Tests unitarios**: Crear tests con `pytest`
3. **Variables de entorno**: Usar `.env` para configuración
4. **Retry logic**: Agregar reintentos automáticos en fallos de red
5. **Validación de datos**: Usar `pydantic` para validar schemas
6. **Scheduling**: Configurar Jobs en Databricks para ejecución automática